In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Change working directory to be current folder
import os
os.chdir('/content/drive/MyDrive/coviddata')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 2020VAERSDATA.csv
 2020VAERSSYMPTOMS.csv
 2020VAERSVAX.csv
 2021VAERSDATA.csv
 2021VAERSSYMPTOMS.csv
 2021VAERSVAX.csv
 bert_model.h5
 BERT_pr.PNG
 BERT_roc_multi.PNG
 BERT_roc.png
 bert_symptom_weights.h5
 classes_41.txt
 classes.txt
 corrected_hospitalized_symptoms_clean_20.csv
 corrected_non_hospitalized_symptoms_clean_20.csv
 corrected_symptoms_clean_41.csv
'COVID-SVM-Hospitalised Prediction.ipynb'
 data_complete2.csv
 data_complete2r.csv
 data_complete.csv
 hospitalized_classes.txt
 hospitalized_symptoms_clean_20.csv
 hospitalized_symptoms_clean.csv
 hospitalized_symptoms_counts_20.txt
 hospitalized_symptoms_counts.csv
 non_hospitalized_symptoms_clean_20.csv
 non_hospitalized_symptoms_clean.csv
 non_hospitalized_symptoms_counts_20.txt
 non_hospitalized_symptoms_counts.csv
 nonzero_hospital_21.csv
 nonzero_non_hospital_20.csv
 predictions.txt
'PRSPM BERT

# Import libraries

In [2]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import tensorflow as tf
import os

print("Versions of key libraries")
print("---")
print("tensorflow: ", tf.__version__)
print("numpy:      ", np.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)

Versions of key libraries
---
tensorflow:  2.7.0
numpy:       1.19.5
matplotlib:  3.2.2
sklearn:     0.22.2.post1


In [3]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 28.5 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [5]:
import pandas as pd
from transformers import TFBertModel
seed_value = 42
import os
os.environ['PYTHONHASHSEED'] = str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
np.set_printoptions(precision=2)
import tensorflow as tf
tf.random.set_seed(seed_value)
import tensorflow_addons as tfa
import keras
import keras.layers as layers
from keras.callbacks import ModelCheckpoint
import transformers
from tqdm import tqdm
import seaborn as sns
from transformers import BertTokenizer

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.4.0 and strictly below 2.7.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


# BERT text preprocessing

In [6]:
#fn takes 1 input which is free text and returns tokenized form for model inference
def prepare_bert_input(sentences, seq_len = 150, bert_name = 'bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(bert_name)
    encodings = tokenizer(sentences.tolist(), truncation=True, padding='max_length',
                                max_length=seq_len)
    input = [np.array(encodings["input_ids"]), np.array(encodings["token_type_ids"]),
               np.array(encodings["attention_mask"])]
    return input

# Load BERT model + weights + classes

In [7]:
def create_model(MAX_SEQ_LEN = 150, BERT_NAME = 'bert-base-uncased', N_CLASSES = 41):
    input_ids = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input_ids')
    input_type = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='token_type_ids')
    input_mask = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='attention_mask')
    inputs = [input_ids, input_type, input_mask]
    bert = TFBertModel.from_pretrained(BERT_NAME)
    bert_outputs = bert(inputs)
    last_hidden_states = bert_outputs.last_hidden_state
    avg = layers.GlobalAveragePooling1D()(last_hidden_states)
    output = layers.Dense(N_CLASSES, activation="sigmoid")(avg)
    model = keras.Model(inputs=inputs, outputs=output)
    model.summary()
    return model

In [8]:
#fn to load model and weights
def load_model(weights):
    model = create_model()
    loss = keras.losses.BinaryCrossentropy()
    best_weights_file = weights
    model.load_weights(best_weights_file)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=3e-5)
    model.compile(loss=loss, optimizer=opt, metrics=[keras.metrics.AUC(multi_label=True, curve="ROC", num_labels= 41),
                                                     keras.metrics.BinaryAccuracy()])
    return model

In [9]:
#place filepath for weights file below
weights = "bert_symptom_weights.h5" 

In [10]:
# Unpickling and load classes list
import pickle
#place filepath for classes_41 file below
with open("/content/drive/MyDrive/coviddata/classes_41.txt", "rb") as fp: 
  classes_41 = pickle.load(fp)
print('classes_41 txt loaded!')

classes_41 txt loaded!


In [11]:
model = load_model(weights)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 150)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 150)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 150)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'token_type_ids[0][0]',     

# Inference

In [51]:
#place user input into input1,  can add on other inputs as input2, etc.
input1 = 'Had my blood taken recently, vomitted a lot in the last few days and am dizzy.'

sentences = np.asarray([input1])

token = prepare_bert_input(sentences)
prediction = model.predict(token)
prediction

array([[0.01, 0.02, 0.  , 0.02, 0.  , 0.  , 0.38, 0.07, 0.03, 0.33, 0.03,
        0.99, 0.01, 0.82, 0.01, 0.  , 0.04, 0.19, 0.03, 0.01, 0.  , 0.01,
        0.01, 0.  , 0.  , 0.01, 0.01, 0.  , 0.03, 0.  , 0.03, 0.01, 0.02,
        0.01, 0.01, 0.  , 0.02, 0.01, 0.02, 0.01, 0.02]], dtype=float32)

In [52]:
symptomlist = []
# report n number of closest matching symptoms, set to 4
topn = 4
for sentence, pred in zip(sentences, prediction):
    pred = pred.tolist()
    index = np.argsort(pred)
    index = list(index[-topn:])
    for i in index:
      symptom = classes_41[i]
      symptomlist.append(symptom)


symptomlist

['SYMPTOMScomputerised tomogram',
 'SYMPTOMSblood test',
 'SYMPTOMSvomiting',
 'SYMPTOMSdizziness']

In [53]:
symptomlist = []
# report symptoms with probability above 0.3
thresh = 0.2

for sentence, pred in zip(sentences, prediction):
    pred = pred.tolist()
    for score in pred:
      if score>thresh:
        i = pred.index(score)
        symptom = classes_41[i]
        symptomlist.append(symptom)


symptomlist

['SYMPTOMSblood test',
 'SYMPTOMScomputerised tomogram',
 'SYMPTOMSdizziness',
 'SYMPTOMSvomiting']

In [54]:
#place user input into input1,  can add on other inputs as input2, etc.
input2 = 'I am having a headache.'

sentences = np.asarray([input2])

token = prepare_bert_input(sentences)
prediction = model.predict(token)
prediction

array([[0.02, 0.01, 0.  , 0.89, 0.01, 0.01, 0.08, 0.03, 0.01, 0.19, 0.05,
        0.01, 0.01, 0.01, 0.  , 0.01, 0.01, 0.06, 0.03, 0.  , 0.  , 0.02,
        0.03, 0.05, 0.01, 0.05, 0.05, 0.01, 0.1 , 0.01, 0.02, 0.05, 0.01,
        0.05, 0.02, 0.  , 0.09, 0.02, 0.04, 0.01, 0.01]], dtype=float32)

In [55]:
symptomlist = []
# report symptoms with probability above 0.5
thresh = 0.3

for sentence, pred in zip(sentences, prediction):
    pred = pred.tolist()
    for score in pred:
      if score>thresh:
        i = pred.index(score)
        symptom = classes_41[i]
        symptomlist.append(symptom)


symptomlist

In [ ]:
''''
symptomlist = []
for sentence, pred in zip(sentences, prediction):
    mask = (pred > 0.35).astype(bool)
    print(mask)
    symptom = list(classes_41[mask]) if any(mask) else "SYMPTOMSnull"
    symptomlist.append(symptom)

symptomlist
''''

[False False False False False False False False False False False False
 False False False False False False False False False  True  True False
 False False False False False False False False False False False False
 False False False False False]


[['SYMPTOMSinjection site pain', 'SYMPTOMSarthralgia']]